In [56]:
import torch
import torchvision
import torch.nn as nn
import time
import json
import datetime
import matplotlib.pyplot as plt

In [36]:
%cd

/home/kacper


In [114]:
class RecurrentSimple(nn.Module):
    def __init__(
        self,
        input_size, # features
        hidden_size,
        num_layers,
        batch_size, # time (?)
        num_classes,
        avgpool_dim = 32,
        add_dropout=True
    ):
        super(RecurrentSimple, self).__init__()
        
        self.normalization = nn.BatchNorm1d(input_size)
        self.features = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.avgpool = nn.AdaptiveAvgPool1d(avgpool_dim)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size * avgpool_dim, 512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, num_classes),
        )
        
        self.init_hidden_shape = (num_layers, batch_size, hidden_size)

    def forward(self, x, 
                #hidden
               ):
        x = self.normalization(x)
        x = x.mT
        x, hidden = self.features(x, 
                                  #hidden
                                 )
        x = x.mT
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return (
            x,
            #hidden
        )
    
    def init_hidden(self, device="cpu"):
        return torch.randn(*self.init_hidden_shape).to(device)

In [105]:
from DeepLearning.Project2.data_loading_preparation import load_audio_dataloaders_validation
batch_size = 128
train, test, val = load_audio_dataloaders_validation(bs=batch_size, limit_11=0.0)
# torch.Size([128, 32, 20])


In [118]:
def eval_accuracy(model, dataloader, reccurrent, training_device='cpu'):
    with torch.no_grad():
        model.to(training_device)
        correct = 0
        all_so_far = 0
        for inputs, labels in dataloader:
            inputs, labels = inputs.float().to(training_device), labels.float().to(training_device)
            yhat = model(inputs, 
                         #reccurrent
                        )
            pred = torch.argmax(yhat[0], dim=1)

            
            all_so_far += labels.size().numel()
            correct += torch.sum(pred.eq(labels)).item()
    return correct/all_so_far

In [126]:
device = "cuda"
criterion_weights = torch.tensor([1.] * 11 + [0.0]).to(device)
hidden_size = 16
input_size = 20 # features
num_layers = 1
time_size = 32 # time (?)


model = RecurrentSimple(input_size, hidden_size, num_layers, batch_size, 12).to(device)


criterion = nn.CrossEntropyLoss(weight=criterion_weights)
# https://deepdatascience.wordpress.com/2016/11/18/which-lstm-optimizer-to-use/
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=[0.9, 0.999], eps=10e-8)
#hidden_prev = model.init_hidden(device)


for epoch in range(1, 51):
    for x, y in train:
        optimizer.zero_grad()
        x, y = x.float().to(device), y.float().to(device)
                
        
        (yhat, 
         #hidden
        ) = model(x, 
                  #hidden_prev
                 )
        #hidden_prev = hidden.detach()
        
        loss = criterion(yhat.softmax(1), y.long())
        loss.backward()
        optimizer.step()
    
    val_accuracy = eval_accuracy(model, val, hidden, "cuda")
    
    print(f"Epoch: {epoch}, Loss: {loss.item():.2f}, Accuracy (validation): {val_accuracy:.5f}" )


Epoch: 1, Loss: 2.35, Accuracy (validation): 0.25930
Epoch: 2, Loss: 2.27, Accuracy (validation): 0.35581
Epoch: 3, Loss: 2.25, Accuracy (validation): 0.38605
Epoch: 4, Loss: 2.22, Accuracy (validation): 0.41473
Epoch: 5, Loss: 2.19, Accuracy (validation): 0.46124
Epoch: 6, Loss: 2.23, Accuracy (validation): 0.46744
Epoch: 7, Loss: 2.16, Accuracy (validation): 0.50620
Epoch: 8, Loss: 2.17, Accuracy (validation): 0.49496
Epoch: 9, Loss: 2.14, Accuracy (validation): 0.51938
Epoch: 10, Loss: 2.09, Accuracy (validation): 0.51938
Epoch: 11, Loss: 2.09, Accuracy (validation): 0.52674
Epoch: 12, Loss: 2.06, Accuracy (validation): 0.52868
Epoch: 13, Loss: 2.04, Accuracy (validation): 0.54031
Epoch: 14, Loss: 2.06, Accuracy (validation): 0.55078
Epoch: 15, Loss: 2.05, Accuracy (validation): 0.54574
Epoch: 16, Loss: 2.00, Accuracy (validation): 0.54961
Epoch: 17, Loss: 2.00, Accuracy (validation): 0.54767
Epoch: 18, Loss: 2.02, Accuracy (validation): 0.55853
Epoch: 19, Loss: 2.00, Accuracy (vali

In [127]:
device = "cuda"
criterion_weights = torch.tensor([1.] * 11 + [0.0]).to(device)
hidden_size = 32
input_size = 20 # features
num_layers = 1
time_size = 32 # time (?)


model = RecurrentSimple(input_size, hidden_size, num_layers, batch_size, 12).to(device)


criterion = nn.CrossEntropyLoss(weight=criterion_weights)
# https://deepdatascience.wordpress.com/2016/11/18/which-lstm-optimizer-to-use/
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=[0.9, 0.999], eps=10e-8)
#hidden_prev = model.init_hidden(device)


for epoch in range(1, 51):
    for x, y in train:
        optimizer.zero_grad()
        x, y = x.float().to(device), y.float().to(device)
                
        
        (yhat, 
         #hidden
        ) = model(x, 
                  #hidden_prev
                 )
        #hidden_prev = hidden.detach()
        
        loss = criterion(yhat.softmax(1), y.long())
        loss.backward()
        optimizer.step()
    
    val_accuracy = eval_accuracy(model, val, hidden, "cuda")
    
    print(f"Epoch: {epoch}, Loss: {loss.item():.2f}, Accuracy (validation): {val_accuracy:.5f}" )


Epoch: 1, Loss: 2.29, Accuracy (validation): 0.28023
Epoch: 2, Loss: 2.34, Accuracy (validation): 0.37946
Epoch: 3, Loss: 2.18, Accuracy (validation): 0.43101
Epoch: 4, Loss: 2.13, Accuracy (validation): 0.45310
Epoch: 5, Loss: 2.07, Accuracy (validation): 0.48295
Epoch: 6, Loss: 2.11, Accuracy (validation): 0.49109
Epoch: 7, Loss: 2.09, Accuracy (validation): 0.51008
Epoch: 8, Loss: 2.15, Accuracy (validation): 0.52519
Epoch: 9, Loss: 2.05, Accuracy (validation): 0.52636
Epoch: 10, Loss: 2.04, Accuracy (validation): 0.53450
Epoch: 11, Loss: 2.04, Accuracy (validation): 0.53178
Epoch: 12, Loss: 1.96, Accuracy (validation): 0.54341
Epoch: 13, Loss: 1.98, Accuracy (validation): 0.55969
Epoch: 14, Loss: 1.93, Accuracy (validation): 0.57209
Epoch: 15, Loss: 1.95, Accuracy (validation): 0.55271
Epoch: 16, Loss: 1.94, Accuracy (validation): 0.56860
Epoch: 17, Loss: 1.95, Accuracy (validation): 0.56628
Epoch: 18, Loss: 1.96, Accuracy (validation): 0.58682
Epoch: 19, Loss: 1.94, Accuracy (vali

In [128]:
device = "cuda"
criterion_weights = torch.tensor([1.] * 11 + [0.0]).to(device)
hidden_size = 64
input_size = 20 # features
num_layers = 1
time_size = 32 # time (?)


model = RecurrentSimple(input_size, hidden_size, num_layers, batch_size, 12).to(device)


criterion = nn.CrossEntropyLoss(weight=criterion_weights)
# https://deepdatascience.wordpress.com/2016/11/18/which-lstm-optimizer-to-use/
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=[0.9, 0.999], eps=10e-8)
#hidden_prev = model.init_hidden(device)


for epoch in range(1, 51):
    for x, y in train:
        optimizer.zero_grad()
        x, y = x.float().to(device), y.float().to(device)
                
        
        (yhat, 
         #hidden
        ) = model(x, 
                  #hidden_prev
                 )
        #hidden_prev = hidden.detach()
        
        loss = criterion(yhat.softmax(1), y.long())
        loss.backward()
        optimizer.step()
    
    val_accuracy = eval_accuracy(model, val, hidden, "cuda")
    
    print(f"Epoch: {epoch}, Loss: {loss.item():.2f}, Accuracy (validation): {val_accuracy:.5f}" )


Epoch: 1, Loss: 2.35, Accuracy (validation): 0.29845
Epoch: 2, Loss: 2.26, Accuracy (validation): 0.37403
Epoch: 3, Loss: 2.21, Accuracy (validation): 0.44496
Epoch: 4, Loss: 2.21, Accuracy (validation): 0.45543
Epoch: 5, Loss: 2.16, Accuracy (validation): 0.46279
Epoch: 6, Loss: 2.19, Accuracy (validation): 0.49070
Epoch: 7, Loss: 2.14, Accuracy (validation): 0.50504
Epoch: 8, Loss: 2.06, Accuracy (validation): 0.51047
Epoch: 9, Loss: 2.12, Accuracy (validation): 0.53333
Epoch: 10, Loss: 2.13, Accuracy (validation): 0.54186
Epoch: 11, Loss: 2.08, Accuracy (validation): 0.54574
Epoch: 12, Loss: 2.06, Accuracy (validation): 0.56124
Epoch: 13, Loss: 2.05, Accuracy (validation): 0.55891
Epoch: 14, Loss: 2.07, Accuracy (validation): 0.56705
Epoch: 15, Loss: 2.03, Accuracy (validation): 0.56938
Epoch: 16, Loss: 2.06, Accuracy (validation): 0.56860
Epoch: 17, Loss: 2.07, Accuracy (validation): 0.56357
Epoch: 18, Loss: 2.00, Accuracy (validation): 0.56667
Epoch: 19, Loss: 1.96, Accuracy (vali

In [129]:
device = "cuda"
criterion_weights = torch.tensor([1.] * 11 + [0.0]).to(device)
hidden_size = 16
input_size = 20 # features
num_layers = 2
time_size = 32 # time (?)


model = RecurrentSimple(input_size, hidden_size, num_layers, batch_size, 12).to(device)


criterion = nn.CrossEntropyLoss(weight=criterion_weights)
# https://deepdatascience.wordpress.com/2016/11/18/which-lstm-optimizer-to-use/
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=[0.9, 0.999], eps=10e-8)
#hidden_prev = model.init_hidden(device)


for epoch in range(1, 51):
    for x, y in train:
        optimizer.zero_grad()
        x, y = x.float().to(device), y.float().to(device)
                
        
        (yhat, 
         #hidden
        ) = model(x, 
                  #hidden_prev
                 )
        #hidden_prev = hidden.detach()
        
        loss = criterion(yhat.softmax(1), y.long())
        loss.backward()
        optimizer.step()
    
    val_accuracy = eval_accuracy(model, val, hidden, "cuda")
    
    print(f"Epoch: {epoch}, Loss: {loss.item():.2f}, Accuracy (validation): {val_accuracy:.5f}" )


Epoch: 1, Loss: 2.34, Accuracy (validation): 0.27326
Epoch: 2, Loss: 2.26, Accuracy (validation): 0.36395
Epoch: 3, Loss: 2.22, Accuracy (validation): 0.40620
Epoch: 4, Loss: 2.16, Accuracy (validation): 0.44380
Epoch: 5, Loss: 2.17, Accuracy (validation): 0.46667
Epoch: 6, Loss: 2.19, Accuracy (validation): 0.47597
Epoch: 7, Loss: 2.13, Accuracy (validation): 0.50116
Epoch: 8, Loss: 2.14, Accuracy (validation): 0.51783
Epoch: 9, Loss: 2.16, Accuracy (validation): 0.51434
Epoch: 10, Loss: 2.09, Accuracy (validation): 0.53372
Epoch: 11, Loss: 2.16, Accuracy (validation): 0.54147
Epoch: 12, Loss: 2.16, Accuracy (validation): 0.52829
Epoch: 13, Loss: 2.09, Accuracy (validation): 0.54574
Epoch: 14, Loss: 2.08, Accuracy (validation): 0.54264
Epoch: 15, Loss: 2.10, Accuracy (validation): 0.55155
Epoch: 16, Loss: 2.03, Accuracy (validation): 0.55814
Epoch: 17, Loss: 2.06, Accuracy (validation): 0.57907
Epoch: 18, Loss: 1.99, Accuracy (validation): 0.57248
Epoch: 19, Loss: 1.94, Accuracy (vali

In [130]:
device = "cuda"
criterion_weights = torch.tensor([1.] * 11 + [0.0]).to(device)
hidden_size = 32
input_size = 20 # features
num_layers = 2
time_size = 32 # time (?)


model = RecurrentSimple(input_size, hidden_size, num_layers, batch_size, 12).to(device)


criterion = nn.CrossEntropyLoss(weight=criterion_weights)
# https://deepdatascience.wordpress.com/2016/11/18/which-lstm-optimizer-to-use/
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=[0.9, 0.999], eps=10e-8)
#hidden_prev = model.init_hidden(device)


for epoch in range(1, 51):
    for x, y in train:
        optimizer.zero_grad()
        x, y = x.float().to(device), y.float().to(device)
                
        
        (yhat, 
         #hidden
        ) = model(x, 
                  #hidden_prev
                 )
        #hidden_prev = hidden.detach()
        
        loss = criterion(yhat.softmax(1), y.long())
        loss.backward()
        optimizer.step()
    
    val_accuracy = eval_accuracy(model, val, hidden, "cuda")
    
    print(f"Epoch: {epoch}, Loss: {loss.item():.2f}, Accuracy (validation): {val_accuracy:.5f}" )


Epoch: 1, Loss: 2.35, Accuracy (validation): 0.30039
Epoch: 2, Loss: 2.27, Accuracy (validation): 0.39380
Epoch: 3, Loss: 2.19, Accuracy (validation): 0.42481
Epoch: 4, Loss: 2.17, Accuracy (validation): 0.45465
Epoch: 5, Loss: 2.22, Accuracy (validation): 0.46434
Epoch: 6, Loss: 2.23, Accuracy (validation): 0.48682
Epoch: 7, Loss: 2.19, Accuracy (validation): 0.48915
Epoch: 8, Loss: 2.16, Accuracy (validation): 0.51550
Epoch: 9, Loss: 2.17, Accuracy (validation): 0.52403
Epoch: 10, Loss: 2.06, Accuracy (validation): 0.54147
Epoch: 11, Loss: 2.10, Accuracy (validation): 0.53798
Epoch: 12, Loss: 2.11, Accuracy (validation): 0.55271
Epoch: 13, Loss: 2.05, Accuracy (validation): 0.55155
Epoch: 14, Loss: 2.04, Accuracy (validation): 0.56705
Epoch: 15, Loss: 2.05, Accuracy (validation): 0.56163
Epoch: 16, Loss: 2.06, Accuracy (validation): 0.57752
Epoch: 17, Loss: 2.01, Accuracy (validation): 0.56085
Epoch: 18, Loss: 2.04, Accuracy (validation): 0.58217
Epoch: 19, Loss: 2.04, Accuracy (vali

In [131]:
device = "cuda"
criterion_weights = torch.tensor([1.] * 11 + [0.0]).to(device)
hidden_size = 64
input_size = 20 # features
num_layers = 2
time_size = 32 # time (?)


model = RecurrentSimple(input_size, hidden_size, num_layers, batch_size, 12).to(device)


criterion = nn.CrossEntropyLoss(weight=criterion_weights)
# https://deepdatascience.wordpress.com/2016/11/18/which-lstm-optimizer-to-use/
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=[0.9, 0.999], eps=10e-8)
#hidden_prev = model.init_hidden(device)


for epoch in range(1, 51):
    for x, y in train:
        optimizer.zero_grad()
        x, y = x.float().to(device), y.float().to(device)
                
        
        (yhat) = model(x)
        
        loss = criterion(yhat.softmax(1), y.long())
        loss.backward()
        optimizer.step()
    
    val_accuracy = eval_accuracy(model, val, hidden, "cuda")
    
    print(f"Epoch: {epoch}, Loss: {loss.item():.2f}, Accuracy (validation): {val_accuracy:.5f}" )


Epoch: 1, Loss: 2.29, Accuracy (validation): 0.28915
Epoch: 2, Loss: 2.31, Accuracy (validation): 0.36860
Epoch: 3, Loss: 2.26, Accuracy (validation): 0.37984
Epoch: 4, Loss: 2.26, Accuracy (validation): 0.42326
Epoch: 5, Loss: 2.23, Accuracy (validation): 0.41202
Epoch: 6, Loss: 2.23, Accuracy (validation): 0.40853
Epoch: 7, Loss: 2.18, Accuracy (validation): 0.41395
Epoch: 8, Loss: 2.17, Accuracy (validation): 0.43643
Epoch: 9, Loss: 2.17, Accuracy (validation): 0.46279
Epoch: 10, Loss: 2.13, Accuracy (validation): 0.47209
Epoch: 11, Loss: 2.23, Accuracy (validation): 0.46512
Epoch: 12, Loss: 2.26, Accuracy (validation): 0.45116
Epoch: 13, Loss: 2.13, Accuracy (validation): 0.42326
Epoch: 14, Loss: 2.18, Accuracy (validation): 0.43566
Epoch: 15, Loss: 2.21, Accuracy (validation): 0.46783
Epoch: 16, Loss: 2.15, Accuracy (validation): 0.47907
Epoch: 17, Loss: 2.22, Accuracy (validation): 0.43488
Epoch: 18, Loss: 2.20, Accuracy (validation): 0.45310
Epoch: 19, Loss: 2.21, Accuracy (vali